In [ ]:
import turtle
import numpy as np
import tkinter as tk

In [ ]:
# root = tk.Tk()

# canvas = turtle.ScrolledCanvas(root)
# canvas.pack(side=tk.LEFT)

# screen = turtle.TurtleScreen(canvas)
# screen.setworldcoordinates(-10, 100, 100, -10)

# turtle = turtle.RawTurtle(screen)
# turtle.goto(90, 90)

# screen.mainloop()

In [ ]:
# Create the game window
game_window = turtle.Screen()
game_window.title("RL@Tahseen")
game_window.setup(width=800,height=600)
game_window.bgcolor("black")
game_window.setworldcoordinates(0,0,800,600)
# Stop window from updating automatically
game_window.tracer(0)

In [ ]:
# Create the road
out_left_line = turtle.Turtle()
out_left_line.speed(0) # set the animation speed to 0
out_left_line.color("white")
out_left_line.shape("square")
out_left_line.shapesize(stretch_wid = 20,stretch_len = 1)
out_left_line.penup()
out_left_line.goto(100,300)

in_left_line = turtle.Turtle()
in_left_line.speed(0) # set the animation speed to 0
in_left_line.color("white")
in_left_line.shape("square")
in_left_line.shapesize(stretch_wid = 10,stretch_len = 1)
in_left_line.penup()
in_left_line.goto(200,300)

out_right_line = turtle.Turtle()
out_right_line.speed(0) # set the animation speed to 0
out_right_line.color("white")
out_right_line.shape("square")
out_right_line.shapesize(stretch_wid = 20,stretch_len=1)
out_right_line.penup()
out_right_line.goto(700,300)

in_right_line = turtle.Turtle()
in_right_line.speed(0) # set the animation speed to 0
in_right_line.color("white")
in_right_line.shape("square")
in_right_line.shapesize(stretch_wid = 10,stretch_len = 1)
in_right_line.penup()
in_right_line.goto(600,300)

up_in = turtle.Turtle()
up_in.shape("square")
up_in.shapesize(stretch_wid=1,stretch_len=20)
up_in.color("white")
up_in.penup()
up_in.speed(0)
up_in.goto(400,400)

down_in = turtle.Turtle()
down_in.shape("square")
down_in.shapesize(stretch_wid=1,stretch_len=20)
down_in.color("white")
down_in.penup()
down_in.speed(0)
down_in.goto(400,200)

up_out = turtle.Turtle()
up_out.shape("square")
up_out.shapesize(stretch_wid=1,stretch_len=30)
up_out.color("white")
up_out.penup()
up_out.speed(0)
up_out.goto(400,500)

down_out = turtle.Turtle()
down_out.shape("square")
down_out.shapesize(stretch_wid=1,stretch_len=30)
down_out.color("white")
down_out.penup()
down_out.speed(0)
down_out.goto(400,100)

In [ ]:
# Create Agent

agent = turtle.Turtle()
agent.speed(0)
agent.shape("square")
agent.penup()
agent.color("red")
agent.shapesize(stretch_wid = 1, stretch_len=2)
agent.goto(400,150)
# move speed
agent.dx = 1
agent.dy = 1

In [ ]:
# game_window.reset()

# # Get the starting co-ordinate of the agent
start_x = agent.xcor()
start_y = agent.ycor()

# actions
actions = ["up","down","forward","diagonal_forw_up","diagonal_forw_down"]
possible_actions = len(actions)
curr_state = [start_x, start_y]

# set initial q-values of all states. Each state has 5 q-values for 5 actions that can be taken from that state.
q_values = np.zeros((game_window.window_width(), game_window.window_height(),possible_actions))

# set rewards of all states intially to -100
rewards = np.full((game_window.window_width(), game_window.window_height()), -100.)


# Set rewards of all states in the roads initially to -1
rewards[120:200,120:500] = -1.
rewards[620:700,120:500] = -1.
rewards[200:620,120:200] = -1.
rewards[200:620,420:500] = -1.
rewards[240:300,130:200] = 100.

In [ ]:
def choose_next_action(agent_curr_state_x,agent_curr_state_y,epsilon):
    # agent_curr_state_x should be the current row index
    # agent_curr_state_y should be the current column index
    
    # Now select a random number and check if it is greater than epsilon or not
    if np.random.random() < epsilon:
        # return the maximum q-values out of the 5 q-values in the current state.
        # So that from the current state, the agent can select that action with the highest q-value 
        return np.argmax(q_values[agent_curr_state_x,agent_curr_state_y])
    else:
        return np.random.randint(possible_actions) # select an action randomly from possible actions

def get_next_state_location(agent_curr_state_x,agent_curr_state_y,next_action_to_take):
    
    next_state_x, next_state_y = agent_curr_state_x,agent_curr_state_y
    
    if actions[next_action_to_take] == "up":
        next_state_x, next_state_y = agent_curr_state_x, agent_curr_state_y + agent.dy
    elif actions[next_action_to_take] == "down":
        next_state_x, next_state_y = agent_curr_state_x, agent_curr_state_y - agent.dy
    elif actions[next_action_to_take] == "forward":
        next_state_x, next_state_y = agent_curr_state_x + agent.dx, agent_curr_state_y
    elif actions[next_action_to_take] == "diagonal_forw_up":
        next_state_x, next_state_y = agent_curr_state_x + agent.dx, agent_curr_state_y + agent.dy
    elif actions[next_action_to_take] == "diagonal_forw_down":
        next_state_x, next_state_y = agent_curr_state_x + agent.dx, agent_curr_state_y - agent.dy
    else:
        print("Do not move!")
    
    return next_state_x, next_state_y

def is_terminal_state(agent_curr_loc_x, agent_curr_loc_y):
    if rewards[agent_curr_loc_x,agent_curr_loc_y] == -1.:
        return False
    else:
        return True
    

In [ ]:
    # Start Training

    # How many times we will train the agent
    episodes = 1000

    # epsilon indicates the percentage of time when we should choose the best action instead of a random action
    epsilon = 0.7 # 90% time select the best action

    # discount factor for future state's q_values
    discount_factor = 0.4

    learning_rate = 0.8

    for episode in range(episodes):
#         time_steps = 0
        print("Episode:",end=" ")
        print(episode)
        while not is_terminal_state(agent.xcor(),agent.ycor()):

            # Select the action to take from the current state
            next_action_to_take = choose_next_action(agent.xcor(),agent.ycor(),epsilon)

#             print("Next Action:", end=" ")
#             print(next_action_to_take)

            # save the current state x and y coordinates
            curr_state_x, curr_state_y = agent.xcor(), agent.ycor()
#             print("Agent's current location:", end=" ")
#             print(curr_state_x)
#             print(curr_state_y)
            # get the next state x and y coordinates
            # the next state is the state where the agent will go after taking the action indicated by 'next_action_to_take'
            next_state_x, next_state_y = get_next_state_location(curr_state_x,curr_state_y,next_action_to_take)
            
#             if next_state_y >= 600:
#                 next_state_y = 599
#             if next_state_x >= 800:
#                 next_state_x = 799
            
            # Get the reward to go to that next state
            reward = rewards[next_state_x,next_state_y]

            # Now calculate the current q-value of the current state for that 'next_action_to_take' from the current state
            current_q_value = q_values[curr_state_x,curr_state_y,next_action_to_take]

            # Now in the next state, there are 5 actions. So there are 5 q-values in the next state and we know those q-values.
            # We can not say now which action will be choosen from that next state in future, but based on the current q-values
            # of the next state, we can guess what can be the maximum q-value that we can achieve in that next state in future 
            # time step. Let's get that maximum q-value in the next state.

            max_achievable_q_values_in_next_state = np.max(q_values[next_state_x,next_state_y])

            # We do not want to get the full effect of that knowledge to our current state. Let's minimize the effect.

            max_achievable_q_values_in_next_state = discount_factor * max_achievable_q_values_in_next_state

            # Now, let us calculate the overall profit the agent can get by going to that next state

            profit = reward + max_achievable_q_values_in_next_state

            # Let us calculate how the profit is compared to the profit in our curent state
            temporal_difference = profit - current_q_value

            # Let us update the current state profit or in other words, q-value. We do not want to be greedy.  
            # So, we will not learn 100%.
            new_q_value_of_the_current_state_for_the_next_action = current_q_value + (learning_rate * temporal_difference)
            q_values[curr_state_x,curr_state_y,next_action_to_take] = new_q_value_of_the_current_state_for_the_next_action

            # Update agents location now
            agent.setx(next_state_x)
            agent.sety(next_state_y)

        agent.setx(start_x)
        agent.sety(start_y)
        
    print("Training Complete!")

In [ ]:
# Main game loop that will keep the game running, inside the loop we are updatin the scene manually
while True:
    game_window.update()
    
#     agent.setx(agent.xcor()+agent.dx)
#     agent.sety(agent.ycor()+agent.dy)

    current_x, current_y = 400, 150
    
    agent.setx(current_x)
    agent.sety(current_y)
    
    print(current_x, end=" ")
    print(current_y)
    
    while not is_terminal_state(current_x, current_y):
        next_action_to_take = choose_next_action(current_x,current_y,1.)
        next_state_x, next_state_y = get_next_state_location(current_x,current_y,next_action_to_take)
        agent.setx(next_state_x)
        agent.sety(next_state_y)
        print(next_state_x, end=" ")
        print(next_state_y)
        current_x = next_state_x
        current_y = next_state_y

#     # Start Training

#     # How many times we will train the agent
#     episodes = 1000

#     # epsilon indicates the percentage of time when we should choose the best action instead of a random action
#     epsilon = 0.9 # 90% time select the best action

#     # discount factor for future state's q_values
#     discount_factor = 0.9

#     learning_rate = 0.9

#     for episode in range(5):
# #         time_steps = 0
#         print("Episode:",end=" ")
#         print(episode)
#         while not is_terminal_state(agent.xcor(),agent.ycor()):

#             # Select the action to take from the current state
#             next_action_to_take = choose_next_action(agent.xcor(),agent.ycor(),epsilon)

#             print("Next Action:", end=" ")
#             print(next_action_to_take)

#             # save the current state x and y coordinates
#             curr_state_x, curr_state_y = agent.xcor(), agent.ycor()
#             print("Agent's current location:", end=" ")
#             print(curr_state_x)
#             print(curr_state_y)
#             # get the next state x and y coordinates
#             # the next state is the state where the agent will go after taking the action indicated by 'next_action_to_take'
#             next_state_x, next_state_y = get_next_state_location(curr_state_x,curr_state_y,next_action_to_take)
            
# #             if next_state_y >= 600:
# #                 next_state_y = 599
# #             if next_state_x >= 800:
# #                 next_state_x = 799
            
#             # Get the reward to go to that next state
#             reward = rewards[next_state_x,next_state_y]

#             # Now calculate the current q-value of the current state for that 'next_action_to_take' from the current state
#             current_q_value = q_values[curr_state_x,curr_state_y,next_action_to_take]

#             # Now in the next state, there are 5 actions. So there are 5 q-values in the next state and we know those q-values.
#             # We can not say now which action will be choosen from that next state in future, but based on the current q-values
#             # of the next state, we can guess what can be the maximum q-value that we can achieve in that next state in future 
#             # time step. Let's get that maximum q-value in the next state.

#             max_achievable_q_values_in_next_state = np.max(q_values[next_state_x,next_state_y])

#             # We do not want to get the full effect of that knowledge to our current state. Let's minimize the effect.

#             max_achievable_q_values_in_next_state = discount_factor * max_achievable_q_values_in_next_state

#             # Now, let us calculate the overall profit the agent can get by going to that next state

#             profit = reward + max_achievable_q_values_in_next_state

#             # Let us calculate how the profit is compared to the profit in our curent state
#             temporal_difference = profit - current_q_value

#             # Let us update the current state profit or in other words, q-value. We do not want to be greedy.  
#             # So, we will not learn 100%.
#             new_q_value_of_the_current_state_for_the_next_action = current_q_value + (learning_rate * temporal_difference)
#             q_values[curr_state_x,curr_state_y,next_action_to_take] = new_q_value_of_the_current_state_for_the_next_action

#             # Update agents location now
#             agent.setx(next_state_x)
#             agent.sety(next_state_y)

#         agent.setx(start_x)
#         agent.sety(start_y)
        
#     agent.setx(400)
#     agent.sety(150)
